Copyright 2021-2023 @ Shenzhen Bay Laboratory & Peking University & Huawei Technologies Co., Ltd

This code is a part of MindSPONGE: MindSpore Simulation Package tOwards Next Generation molecular modelling.

MindSPONGE is open-source software based on the AI-framework: MindSpore (https://www.mindspore.cn/)

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and limitations under the License.

MindSPONGE tutorial protein 04: Hybrid enhanced sampling MetaITS.

In [1]:
import os
os.environ['GLOG_v'] = '4'
os.environ['MS_JIT_MODULES'] = 'sponge'
from mindspore import context
import sys
sys.path.append('../../../src')
from sponge import Sponge
from sponge import ForceField
from sponge import set_global_units, WithEnergyCell
from sponge import Protein
from sponge import UpdaterMD
from sponge.optimizer import SteepestDescent
from sponge.control import VelocityVerlet
from sponge.callback import WriteH5MD, RunInfo
from sponge.control import Langevin
from sponge.function import VelocityGenerator
from sponge.colvar import Torsion
from sponge.sampling import Metadynamics
from sponge.function import PI

[WARNING] ME(21153:140051537028288,MainProcess):2023-08-15-16:34:38.271.609 [mindspore/common/api.py:897] 'mindspore.ms_function' will be deprecated and removed in a future version. Please use 'mindspore.jit' instead.
[WARNING] ME(21153:140051537028288,MainProcess):2023-08-15-16:34:38.273.349 [mindspore/common/api.py:897] 'mindspore.ms_function' will be deprecated and removed in a future version. Please use 'mindspore.jit' instead.
[WARNING] ME(21153:140051537028288,MainProcess):2023-08-15-16:34:38.274.310 [mindspore/common/api.py:897] 'mindspore.ms_function' will be deprecated and removed in a future version. Please use 'mindspore.jit' instead.
[WARNING] ME(21153:140051537028288,MainProcess):2023-08-15-16:34:38.275.067 [mindspore/common/api.py:897] 'mindspore.ms_function' will be deprecated and removed in a future version. Please use 'mindspore.jit' instead.
[WARNING] ME(21153:140051537028288,MainProcess):2023-08-15-16:34:38.275.883 [mindspore/common/api.py:897] 'mindspore.ms_function

In [2]:
context.set_context(mode=context.GRAPH_MODE)
set_global_units('nm', 'kj/mol')

load PDB file and define CVs

In [3]:
system = Protein(pdb='./case1_addH.pdb', rebuild_hydrogen=False)
energy = ForceField(system, 'AMBER.FF14SB')

phi = Torsion([3, 12, 13, 21])
psi = Torsion([12, 13, 21, 36])

do minimization

In [4]:
min_opt = SteepestDescent(system.trainable_params(), 1e-7)
md = Sponge(system, energy, min_opt, metrics={'phi': phi, 'psi': psi})
run_info = RunInfo(10)
cb_h5md = WriteH5MD(system, './p04_1.h5md', save_freq=10)
md.run(500, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-08-15 16:34:43
[MindSPONGE] Step: 0, E_pot: 5021.522, phi: -3.1408052, psi: 3.1149218
[MindSPONGE] Step: 10, E_pot: 960.6975, phi: 3.1386123, psi: 3.0696745
[MindSPONGE] Step: 20, E_pot: 614.571, phi: 3.1335664, psi: 3.0569959
[MindSPONGE] Step: 30, E_pot: 417.6888, phi: 3.1277804, psi: 3.05628
[MindSPONGE] Step: 40, E_pot: 284.51123, phi: 3.1217692, psi: 3.0593946
[MindSPONGE] Step: 50, E_pot: 185.45404, phi: 3.1157963, psi: 3.0630567
[MindSPONGE] Step: 60, E_pot: 107.68286, phi: 3.1099682, psi: 3.066002
[MindSPONGE] Step: 70, E_pot: 44.551086, phi: 3.1043131, psi: 3.0678315
[MindSPONGE] Step: 80, E_pot: -7.8761597, phi: 3.0988307, psi: 3.0685153
[MindSPONGE] Step: 90, E_pot: -52.160034, phi: 3.09351, psi: 3.0681696
[MindSPONGE] Step: 100, E_pot: -90.0798, phi: 3.0883427, psi: 3.0669641
[MindSPONGE] Step: 110, E_pot: -122.924255, phi: 3.0833232, psi: 3.065074
[MindSPONGE] Step: 120, E_pot: -151.65616, phi: 3.0784502, psi: 3.0626621
[MindSPONGE] 

define MetaD section

In [5]:
metad = Metadynamics(
    colvar=[phi, psi],
    update_pace=10,
    height=2.5,
    sigma=0.05,
    grid_min=-PI,
    grid_max=PI,
    grid_bin=360,
    temperature=300,
    bias_factor=100,
)

Run MetaD sampling

In [6]:
sim = WithEnergyCell(system, energy, bias=metad)
vgen = VelocityGenerator(300)
velocity = vgen(system.shape, system.atom_mass)
opt = UpdaterMD(
    system,
    integrator=VelocityVerlet(system),
    thermostat=Langevin(system, 300),
    time_step=1e-3,
    velocity=velocity
)
md = Sponge(sim, optimizer=opt, metrics={'phi': phi, 'psi': psi})
cb_h5md = WriteH5MD(system, './p04_2.h5md', save_freq=10, write_image=False)
md.run(1000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-08-15 16:35:05
[MindSPONGE] Step: 0, E_pot: -465.43478, E_kin: 229.43347, E_tot: -236.00131, Temperature: 328.506, phi: 3.0291107, psi: 3.0177577
[MindSPONGE] Step: 10, E_pot: -428.97208, E_kin: 182.96736, E_tot: -246.00471, Temperature: 261.9752, phi: -3.0937796, psi: 3.0833538
[MindSPONGE] Step: 20, E_pot: -400.25235, E_kin: 175.21399, E_tot: -225.03836, Temperature: 250.8738, phi: -3.0417027, psi: 3.0481613
[MindSPONGE] Step: 30, E_pot: -433.21484, E_kin: 221.76509, E_tot: -211.44975, Temperature: 317.52634, phi: -2.938805, psi: 2.9409747
[MindSPONGE] Step: 40, E_pot: -423.71866, E_kin: 216.11987, E_tot: -207.59879, Temperature: 309.44342, phi: -2.710033, psi: 2.8453958
[MindSPONGE] Step: 50, E_pot: -442.45178, E_kin: 224.98145, E_tot: -217.47034, Temperature: 322.13153, phi: -2.5085588, psi: 2.7854238
[MindSPONGE] Step: 60, E_pot: -420.6448, E_kin: 194.46205, E_tot: -226.18275, Temperature: 278.43344, phi: -2.4396794, psi: 2.699188
[MindSPONG